# Partición Externa de Datos (Hold-out)
Vamos a comenzar realizando la partición de datos externa para separar el conjunto de datos total en los subconjuntos de entrenamiento (train), y prueba (test).
Dado que estamos ante un problema de series temporales, se requiere mantener la coherencia temporal por lo que no se aplicará un shuffle a los datos.
Además en nuestro caso asegurar que datos de entrenamiento no se localizan entre los datos de test, no sólo se limita a las instancias individuales en sí, sino que dado que nuestro objetivo será predecir si en un cierto año y para un determinado país se está ante una posible situación de pre-crisis, también tenemos que asegurar que no existe información relativa a años posteriores en nuestro conjunto de test ni para el país bajo estudio en concreto ni tampoco para otros paises (Hellwig, K.P. (2021). Predicting Fiscal Crises: A Machine Learning Approach), por lo que particionaremos el dataset a partir de un año concreto manteniendo los valores más cercanos a una distribución de porcentajes igual al 80% para datos de entrenamiento y 20% para datos de prueba.

Un punto importante a tener en cuenta es que en nuestro dataset siempre hay un registro asociado a cada país y año, por lo que podemos calcular el año a partir del cual particionar en base a esta información.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Cargamos nuestro dataset
data = pd.read_csv('../datasets/DatasetLimpio.csv', sep=';', na_values='', decimal=',')

# Calculamos el número del registro a partir del cual particionar
pct_train = 0.8

num_paises = len(np.unique(data['Country']))
print(f'Número Total de Países: {num_paises}')

num_anios = len(np.unique(data['Year']))
print(f'Número Total de Años: {num_anios}')

anio_particion = np.min(data['Year']) + round(num_anios*pct_train)
print(f'Año de Particionado: {anio_particion}')

train = data[data['Year'] < anio_particion]
test = data[data['Year'] >= anio_particion]

print(f'Muestras de Entrenamiento: {train.shape} -- {train.shape[0] * 100 / data.shape[0]:.2f}% del total')
print(f'Muestras de Test: {test.shape} -- {test.shape[0] * 100 / data.shape[0]:.2f}% del total')

etiquetas = ['No Crisis', 'Pre-Crisis', 'Crisis/Post-Crisis']
_, ocurrencias_train = np.unique(train['Crisis'], return_counts=True)
_, ocurrencias_test = np.unique(test['Crisis'], return_counts=True)
print(f'Distribución de Muestras:\n\tEntrenamiento: {etiquetas} - {ocurrencias_train}\n\tTest: {etiquetas} - {ocurrencias_test}')

Número Total de Países: 188
Número Total de Años: 46
Año de Particionado: 2007
Muestras de Entrenamiento: (6956, 105) -- 80.43% del total
Muestras de Test: (1692, 105) -- 19.57% del total
Distribución de Muestras:
	Entrenamiento: ['No Crisis', 'Pre-Crisis', 'Crisis/Post-Crisis'] - [3593 1006 2357]
	Test: ['No Crisis', 'Pre-Crisis', 'Crisis/Post-Crisis'] - [953 208 531]


In [3]:
# Guardo las particiones
import os
if not os.path.exists('../particiones'):
    os.mkdir('../particiones')

train.to_csv('../particiones/train_original.csv', sep=';', na_rep='', decimal=',', index=False)
test.to_csv('../particiones/test_original.csv', sep=';', na_rep='', decimal=',', index=False)